In [5]:
%pwd

'c:\\Users\\Administrator\\OneDrive\\Desktop\\New folder\\Medical-Chatbot-Generative-AI'

In [4]:
import os
os.chdir("../")

In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader,)
    documents=loader.load()
    return documents

In [21]:
extracted_data=load_pdf_file(data="data/")

In [22]:
def split_text(extracted_data):
    splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    chunks=splitter.split_documents(extracted_data)
    return chunks

In [23]:
text_chunks=split_text(extracted_data)
print('lenth of text chunks',len(text_chunks))

lenth of text chunks 5860


In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


c:\Users\Administrator\OneDrive\Desktop\New folder\Medical-Chatbot-Generative-AI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Administrator\OneDrive\Desktop\New folder\Medical-Chatbot-Generative-AI\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support 

In [27]:
query=embeddings.embed_query('hello world')
print('query',len(query))

query 384


In [30]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [37]:
from langchain_pinecone import PineconeVectorStore

doc_storing = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [40]:
doc_search=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,)

In [38]:
retriver=doc_search.as_retriever(
    search_type="similarity", search_kwaegs={"k":3})

query_results=retriver.invoke('what is acne?')

In [39]:
query_results

[Document(id='215bcfeb-451d-47fd-9d6f-4c3e29e595d1', metadata={'page': 39.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='cfb69266-01ed-4337-aa2e-86c2dc955b73', metadata={'page': 38.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='caacebb1-5474-4fb3-9a24-42cba6cc6ba1', metadata={'page': 37.0, 'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when

In [42]:
load_dotenv()
Groq_Api_Key=os.getenv("GROQ_API_KEY")

In [68]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGroq(temperature=0, groq_api_key=Groq_Api_Key, model_name="llama-3.1-70b-versatile")


system_prompt = '''
            you are an AI assistant who's job is to answer the user questions about the medical domain.
            and help them to find the relevant information from the medical documents and retrived from the documents
            if you don't know the answer to the question make a polite response to the user.
            
            INSTRUCTION:
            1.try to answer the user questions with the information from the medical documents don't mention.keep it simple and easy to understand.
            2.if you don't know the answer to the question make a polite response to the user.
            3.then try to ask some follow-up questions to the user to get more information about the question.
            4.make it brief and consice.
            '\n\n'
            "{context}"'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ('human' ,"{input}") ] )


In [69]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

qanda_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,qanda_chain)

In [72]:
response=rag_chain.invoke({"input":'what is acromegaly and gigantism?'})
print(response['answer'])

Acromegaly and gigantism are disorders caused by the abnormal release of a chemical called growth hormone from the pituitary gland in the brain. This leads to increased growth in bone and soft tissue, as well as other disturbances throughout the body.

The main difference between the two is the age at which the abnormality occurs. If it happens before bone growth stops, usually during childhood or adolescence, it's called gigantism, resulting in unusual height. If it occurs after bone growth stops, usually in adulthood, it's called acromegaly.
